<a href="https://colab.research.google.com/github/dangquangminh-minh/TH_TTNT_BaoCao/blob/main/Minimax_v%C3%A0_Alpha_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import math
import numpy as np

# Định nghĩa các hằng số
X = "X"
O = "O"
RONG = None  # Thay EMPTY bằng RONG

nguoi_choi = None
may = None

# ------------------ CÁC HÀM XỬ LÝ GAME ------------------

def khoi_tao_bang():
    """Tạo bảng rỗng 3x3 ban đầu"""
    return [[RONG, RONG, RONG],
            [RONG, RONG, RONG],
            [RONG, RONG, RONG]]

#hàm này giúp xem đang đến lượt của mình hay là ai
def luot_di_hien_tai(bang):
    """Xác định xem đang là lượt của X hay O"""
    dem = 0
    for hang in bang:
        for o in hang:
            if o is not None:
                dem += 1
    # Nếu số nước đi là lẻ -> Lượt AI, chẵn -> Lượt người chơi
    # (Giả sử người chơi chọn X thì X đi trước)
    return may if dem % 2 != 0 else nguoi_choi

def cac_nuoc_di_hop_le(bang):
    """Trả về tập hợp các tọa độ (hàng, cột) còn trống"""
    ket_qua = set() # khởi tạo 1 tập hợp rỗng , dùng list cũng đc nhưng dùng get thì nó logic toán học
    for i in range(3): # duyệt mảng từ 0-2
        for j in range(3):
            if bang[i][j] == RONG:
                ket_qua.add((i, j))
    return ket_qua

def thuc_hien_nuoc_di(bang, hanh_dong):
    """Trả về bảng mới sau khi thực hiện nước đi"""
    bang_moi = copy.deepcopy(bang)
    (i, j) = hanh_dong
    bang_moi[i][j] = luot_di_hien_tai(bang)
    return bang_moi

# ------------------ CÁC HÀM KIỂM TRA CHIẾN THẮNG ------------------

def kiem_tra_hang_ngang(bang):
    for hang in bang:
        if hang[0] is not None and hang[0] == hang[1] == hang[2]:
            return hang[0]
    return None

def kiem_tra_hang_doc(bang):
    for c in range(3):
        if bang[0][c] is not None and bang[0][c] == bang[1][c] == bang[2][c]:
            return bang[0][c]
    return None

def kiem_tra_duong_cheo(bang):
    if bang[0][0] is not None and bang[0][0] == bang[1][1] == bang[2][2]:
        return bang[0][0]
    if bang[0][2] is not None and bang[0][2] == bang[1][1] == bang[2][0]:
        return bang[0][2]
    return None

def nguoi_thang(bang):
    """Trả về người thắng (X hoặc O), hoặc None nếu chưa ai thắng"""
    return (
        kiem_tra_hang_ngang(bang)
        or kiem_tra_hang_doc(bang)
        or kiem_tra_duong_cheo(bang)
    )

def da_ket_thuc(bang):
    """Kiểm tra game đã kết thúc chưa (có người thắng hoặc hòa)"""
    if nguoi_thang(bang) is not None:
        return True
    for hang in bang:
        if RONG in hang:
            return False
    return True

# ------------------ THUẬT TOÁN MINIMAX ------------------

def tinh_diem(bang):
    """Tính điểm: X thắng = 1, O thắng = -1, Hòa = 0"""
    w = nguoi_thang(bang)
    if w == X:
        return 1
    elif w == O:
        return -1
    return 0

def gia_tri_max(trang_thai):
    if da_ket_thuc(trang_thai):
        return tinh_diem(trang_thai)
    v = -math.inf # con ai nó muốn điểm cao nhất nên ban đầu nó giả sử nó đang âm vô cùng nên bất kì nước đi nào cx ok hơn
    for hanh_dong in cac_nuoc_di_hop_le(trang_thai):
        # ai nó thử đi hết các nước , nó sẽ tìm ra số lớn nhất trong gia_tri_min để xem nếu đi nc này mình sẽ bị thiệt hại như nào
        # có nghĩa là đối thủ sẽ dìm hàng giatrimin thấp nhất vd nước đi A: -100 , b :-10 thì bên ta sẽ lấy max để đỡ thiệt hại nhất
        v = max(v, gia_tri_min(thuc_hien_nuoc_di(trang_thai, hanh_dong)))
    return v

#đối thủ muốn ai thua nên điểm càng nhỏ càng tốt
def gia_tri_min(trang_thai):
    if da_ket_thuc(trang_thai):
        return tinh_diem(trang_thai)
    v = math.inf#nó giả sử ai dang điểm cao và muốn dìm hàng
    for hanh_dong in cac_nuoc_di_hop_le(trang_thai):
        #đối thủ sẽ tìm ra làm cho điểm số ai thấp nhất
        v = min(v, gia_tri_max(thuc_hien_nuoc_di(trang_thai, hanh_dong)))
    return v

def thuat_toan_minimax(bang):
    """Hàm chính để AI tính nước đi tối ưu"""
    nguoi_hien_tai = luot_di_hien_tai(bang)

    if nguoi_hien_tai == X:
        gia_tri_tot_nhat = -math.inf
        nuoc_di_tot_nhat = None
        for hanh_dong in cac_nuoc_di_hop_le(bang):#đi thử các ô
            #giả sử đi dô ô 1 thì nó ép mình đc 5đ , gán gia_tri =5 để tạm đó rồi so  sánh tiếp các ô còn lại đến khi nào tìm đc gia_tri lớn nhất trong các nước đi mà nó dìm mình
            gia_tri = gia_tri_min(thuc_hien_nuoc_di(bang, hanh_dong))
            if gia_tri > gia_tri_tot_nhat:
                gia_tri_tot_nhat = gia_tri
                nuoc_di_tot_nhat = hanh_dong
        return nuoc_di_tot_nhat

    else:  # nguoi_hien_tai == O
        gia_tri_tot_nhat = math.inf
        nuoc_di_tot_nhat = None
        for hanh_dong in cac_nuoc_di_hop_le(bang):
            gia_tri = gia_tri_max(thuc_hien_nuoc_di(bang, hanh_dong))
            if gia_tri < gia_tri_tot_nhat:
                gia_tri_tot_nhat = gia_tri
                nuoc_di_tot_nhat = hanh_dong
        return nuoc_di_tot_nhat

# ------------------ CHƯƠNG TRÌNH CHÍNH ------------------
if __name__ == "__main__":
    bang = khoi_tao_bang()

    print("Chọn người chơi (X/O) - X đi trước:")
    nguoi_choi = input().strip().upper()
    # Nếu người chọn X -> Máy là O, ngược lại Máy là X
    may = O if nguoi_choi == X else X

    while True:
        # Kiểm tra kết thúc game
        if da_ket_thuc(bang):
            w = nguoi_thang(bang)
            if w is None:
                print("Trò chơi kết thúc: HÒA.")
            else:
                print(f"Trò chơi kết thúc: {w} Thắng.")
            break

        current = luot_di_hien_tai(bang)

        # Lượt của Máy (AI)
        if current == may:
            print("\nAI đang suy nghĩ...")
            nuoc_di = thuat_toan_minimax(bang)
            bang = thuc_hien_nuoc_di(bang, nuoc_di)
            print(np.array(bang))

        # Lượt của Người chơi
        else:
            print("\nLượt của bạn:")
            try:
                i = int(input("Nhập Hàng (0-2): "))
                j = int(input("Nhập Cột (0-2): "))

                if 0 <= i <= 2 and 0 <= j <= 2:
                    if bang[i][j] == RONG:
                        bang = thuc_hien_nuoc_di(bang, (i, j))
                        print(np.array(bang))
                    else:
                        print("Ô này đã có người đánh, thử lại.")
                else:
                    print("Tọa độ không hợp lệ (phải từ 0-2).")
            except ValueError:
                print("Vui lòng nhập số nguyên.")

Chọn người chơi (X/O) - X đi trước:
x

Lượt của bạn:


KeyboardInterrupt: Interrupted by user

In [ ]:
import math
import time
import random
from IPython.display import clear_output

def kiem_tra_nguoi_thang(ban_co):
    # Các trường hợp thắng theo hàng ngang
    if ban_co[0] == ban_co[1] and ban_co[1] == ban_co[2]:
        return ban_co[0]
    elif ban_co[3] == ban_co[4] and ban_co[4] == ban_co[5]:
        return ban_co[3]
    elif ban_co[6] == ban_co[7] and ban_co[7] == ban_co[8]:
        return ban_co[6]
    # Các trường hợp thắng theo hàng dọc
    elif ban_co[0] == ban_co[3] and ban_co[3] == ban_co[6]:
        return ban_co[0]
    elif ban_co[1] == ban_co[4] and ban_co[4] == ban_co[7]:
        return ban_co[1]
    elif ban_co[2] == ban_co[5] and ban_co[5] == ban_co[8]:
        return ban_co[2]
    # Các trường hợp thắng theo đường chéo
    elif ban_co[0] == ban_co[4] and ban_co[4] == ban_co[8]:
        return ban_co[0]
    elif ban_co[2] == ban_co[4] and ban_co[4] == ban_co[6]:
        return ban_co[2]
    return None

def in_ban_co(ban_co, thong_bao=""):
    """
    In bàn cờ kèm theo thông báo (nếu có)
    """
    clear_output(wait=True)
    print(f'''
    -------------------
      {ban_co[0]} | {ban_co[1]} | {ban_co[2]}
    ---+---+---
      {ban_co[3]} | {ban_co[4]} | {ban_co[5]}
    ---+---+---
      {ban_co[6]} | {ban_co[7]} | {ban_co[8]}
    -------------------
    ''')
    if thong_bao:
        print(f"--> {thong_bao}")

def lay_cac_o_trong(ban_co):
    o_trong = list()
    for cell in ban_co:
        if cell != "X" and cell != "O":
            o_trong.append(cell)
    return o_trong

def minimax(vi_tri, do_sau, alpha, beta, dang_toi_uu_hoa):
    nguoi_thang = kiem_tra_nguoi_thang(vi_tri)
    if nguoi_thang != None:
        #đi càng ít nước ( do_Sau ) thì điểm càng cao => X lợi
        return 10 - do_sau if nguoi_thang == "X" else -10 + do_sau
        # -10+do_sau : O thích điểm thấp , vd thắng ở bước 2 : -10+2=-8=> hạ gục X nhanh nhất


    if len(lay_cac_o_trong(vi_tri)) == 0:
        return 0 #hòa thì trả về

    if dang_toi_uu_hoa: # Lượt X (Muốn điểm cao nhất)
        max_eval = -math.inf
        for cell in lay_cac_o_trong(vi_tri):
            vi_tri[cell - 1] = "X"#thử đi nước cờ
            danh_gia = minimax(vi_tri, do_sau + 1, alpha, beta, False)#giả sử tìm đc nước đi đánh giá 8 điểm
            max_eval = max(max_eval, danh_gia)
            alpha = max(alpha, danh_gia)#gải sử alpha max : tìm đc nước đi 8 điểm
            vi_tri[cell - 1] = cell # đi thử xong rồi trả về lại như chưa từng đi
            if beta <= alpha: # nhưng thằng O nó tìm đc nước đi để dìm mình xuống chỉ còn 5 điểm ( beta ) mà 5 < 8 => hợp lí nên đường này chắc chắn sẽ bị chặn nên suy ra dục luôn
                break
        return max_eval

    else: # Lượt O (Muốn điểm thấp nhất)
        min_eval = +math.inf
        for cell in lay_cac_o_trong(vi_tri):
            vi_tri[cell - 1] = "O"
            danh_gia = minimax(vi_tri, do_sau + 1, alpha, beta, True)
            min_eval = min(min_eval, danh_gia)
            beta = min(beta, danh_gia)
            vi_tri[cell - 1] = cell
            if beta <= alpha:
                break
        return min_eval

def tim_nuoc_di_tot_nhat(ban_co_hien_tai, AI):
    cac_o_trong = lay_cac_o_trong(ban_co_hien_tai)

    # TỐI ƯU HÓA: Nếu bàn cờ còn quá trống (ví dụ nước đầu tiên hoặc thứ 2)
    # Hãy đánh vào giữa (5) hoặc góc để tiết kiệm thời gian tính toán
    if len(cac_o_trong) >= 8:
        if 5 in cac_o_trong:
            return 5
        else:
            # Chọn ngẫu nhiên một góc (1, 3, 7, 9)
            goc = [x for x in [1, 3, 7, 9] if x in cac_o_trong]
            if goc: return random.choice(goc)

    gia_tri_tot_nhat = -math.inf if AI == "X" else +math.inf
    nuoc_di_tot_nhat = -1

    for cell in cac_o_trong:
        ban_co_hien_tai[cell - 1] = AI
        # Nếu AI là X -> lượt sau là O (Minimizing -> False)
        # Nếu AI là O -> lượt sau là X (Maximizing -> True)
        is_maximizing = False if AI == "X" else True

        gia_tri_nuoc_di = minimax(ban_co_hien_tai, 0, -math.inf, +math.inf, is_maximizing)

        ban_co_hien_tai[cell - 1] = cell # Hoàn tác

        if (AI == "X" and gia_tri_nuoc_di > gia_tri_tot_nhat):
            gia_tri_tot_nhat = gia_tri_nuoc_di
            nuoc_di_tot_nhat = cell
        elif (AI == "O" and gia_tri_nuoc_di < gia_tri_tot_nhat):
            gia_tri_tot_nhat = gia_tri_nuoc_di
            nuoc_di_tot_nhat = cell

    return nuoc_di_tot_nhat

def main():
    while True:
        nguoi_choi = input("Bạn muốn chơi quân nào (X hoặc O)? ").strip().upper()
        if nguoi_choi in ["X", "O"]:
            break
        print("Vui lòng chỉ nhập X hoặc O.")

    AI = "O" if nguoi_choi == "X" else "X"
    print(f"Bạn chọn {nguoi_choi}, Máy sẽ chơi {AI}")
    time.sleep(1)

    van_dau = [*range(1, 10)]
    luot_di = "X"
    dem_nuoc_di = 0

    # In bàn cờ lần đầu
    in_ban_co(van_dau, "Bắt đầu game!")
    time.sleep(0.5)

    while True:
        # Kiểm tra kết quả TRƯỚC khi đi (đề phòng bàn cờ đầy hoặc thắng ngay sau nước đi trước)
        winner = kiem_tra_nguoi_thang(van_dau)
        if winner:
            in_ban_co(van_dau, f"CHÚC MỪNG! '{winner}' ĐÃ CHIẾN THẮNG!!!")
            break
        if dem_nuoc_di == 9:
            in_ban_co(van_dau, "HẾT CỜ! Trò chơi kết thúc với tỉ số HÒA.")
            break

        if luot_di == AI:
            # Lượt Máy
            in_ban_co(van_dau, f"Máy ({AI}) đang suy nghĩ...")

            # Tính toán nước đi
            o_chon = tim_nuoc_di_tot_nhat(van_dau, AI)

            van_dau[o_chon - 1] = AI
            dem_nuoc_di += 1
            luot_di = nguoi_choi

            # In lại bàn cờ để người chơi thấy máy vừa đi đâu
            in_ban_co(van_dau, f"Máy vừa đánh vào ô {o_chon}. Đến lượt bạn!")

        elif luot_di == nguoi_choi:
            # Lượt Người
            # Không gọi in_ban_co ở đây nữa vì đã in ở cuối lượt máy rồi
            # giúp tránh việc màn hình nháy liên tục làm mất ô input
            while True:
                try:
                    user_inp = input(f"Nhập số ô bạn ({nguoi_choi}) muốn đánh (1-9): ")
                    nhap_lieu = int(user_inp.strip())

                    if nhap_lieu in van_dau:
                        van_dau[nhap_lieu - 1] = nguoi_choi
                        dem_nuoc_di += 1
                        luot_di = AI
                        break
                    else:
                        print("Ô này không hợp lệ hoặc đã được đánh. Chọn lại đi!")
                except ValueError:
                    print("Lỗi: Vui lòng nhập một con số nguyên.")

if __name__ == "__main__":
    main()


    -------------------
      O | O | 3
    ---+---+---
      4 | X | 6
    ---+---+---
      7 | X | 9
    -------------------
    
--> Máy vừa đánh vào ô 2. Đến lượt bạn!


KeyboardInterrupt: Interrupted by user